In [39]:
import pandas as pd

path = "./input/" # 路径
train = pd.read_csv(path + "train_set.csv")
test = pd.read_csv(path + "test_set.csv")
train.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [40]:
test.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,25318,51,housemaid,married,unknown,no,174,no,no,telephone,29,jul,308,3,-1,0,unknown
1,25319,32,management,married,tertiary,no,6059,yes,no,cellular,20,nov,110,2,-1,0,unknown
2,25320,60,retired,married,primary,no,0,no,no,telephone,30,jul,130,3,-1,0,unknown
3,25321,32,student,single,tertiary,no,64,no,no,cellular,30,jun,598,4,105,5,failure
4,25322,41,housemaid,married,secondary,no,0,yes,yes,cellular,15,jul,368,4,-1,0,unknown


In [41]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10852 entries, 0 to 10851
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         10852 non-null  int64 
 1   age        10852 non-null  int64 
 2   job        10852 non-null  object
 3   marital    10852 non-null  object
 4   education  10852 non-null  object
 5   default    10852 non-null  object
 6   balance    10852 non-null  int64 
 7   housing    10852 non-null  object
 8   loan       10852 non-null  object
 9   contact    10852 non-null  object
 10  day        10852 non-null  int64 
 11  month      10852 non-null  object
 12  duration   10852 non-null  int64 
 13  campaign   10852 non-null  int64 
 14  pdays      10852 non-null  int64 
 15  previous   10852 non-null  int64 
 16  poutcome   10852 non-null  object
dtypes: int64(8), object(9)
memory usage: 1.4+ MB


In [42]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25317 entries, 0 to 25316
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         25317 non-null  int64 
 1   age        25317 non-null  int64 
 2   job        25317 non-null  object
 3   marital    25317 non-null  object
 4   education  25317 non-null  object
 5   default    25317 non-null  object
 6   balance    25317 non-null  int64 
 7   housing    25317 non-null  object
 8   loan       25317 non-null  object
 9   contact    25317 non-null  object
 10  day        25317 non-null  int64 
 11  month      25317 non-null  object
 12  duration   25317 non-null  int64 
 13  campaign   25317 non-null  int64 
 14  pdays      25317 non-null  int64 
 15  previous   25317 non-null  int64 
 16  poutcome   25317 non-null  object
 17  y          25317 non-null  int64 
dtypes: int64(9), object(9)
memory usage: 3.5+ MB


In [43]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from scipy import sparse
import lightgbm as lgb

# test增加y列
test["y"] = -1

# 类别特征
cate_features = ['job', 'marital','education','default','housing','loan','contact','month','poutcome']
# 数值特征
num_features = ['age', 'balance','day','duration','campaign','pdays','previous']

feature1 = cate_features + num_features
data = pd.concat([train,test])

In [44]:
data.shape

(36169, 18)

In [45]:
# reste_index 重置索引，原本的index变为一列数据，drop = True 不保留原来的index
# pandas Groupby https://zhuanlan.zhihu.com/p/101284491
# pandas Map, apply, applymap  https://zhuanlan.zhihu.com/p/100064394
# pandas Merge https://zhuanlan.zhihu.com/p/102274476

def feat_count(df, df_feature, fe, value, name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].count()).reset_index()
#     print(df_count)
    if not name:
        df_count.columns = fe + [value + "_%s_count" % ("_".join(fe[0]))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on = fe, how = "left") #. fillea(0)
    return df, df_count.columns[1]

for df in feature1:
    data = feat_count(data, data, ["ID"], df)[0]

data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,contact_I_D_count,month_I_D_count,poutcome_I_D_count,age_I_D_count,balance_I_D_count,day_I_D_count,duration_I_D_count,campaign_I_D_count,pdays_I_D_count,previous_I_D_count
0,1,43,management,married,tertiary,no,291,yes,no,unknown,...,1,1,1,1,1,1,1,1,1,1
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,...,1,1,1,1,1,1,1,1,1,1
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,...,1,1,1,1,1,1,1,1,1,1
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,...,1,1,1,1,1,1,1,1,1,1
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,...,1,1,1,1,1,1,1,1,1,1


In [46]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36169 entries, 0 to 36168
Data columns (total 34 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   36169 non-null  int64 
 1   age                  36169 non-null  int64 
 2   job                  36169 non-null  object
 3   marital              36169 non-null  object
 4   education            36169 non-null  object
 5   default              36169 non-null  object
 6   balance              36169 non-null  int64 
 7   housing              36169 non-null  object
 8   loan                 36169 non-null  object
 9   contact              36169 non-null  object
 10  day                  36169 non-null  int64 
 11  month                36169 non-null  object
 12  duration             36169 non-null  int64 
 13  campaign             36169 non-null  int64 
 14  pdays                36169 non-null  int64 
 15  previous             36169 non-null  int64 
 16  pout

In [47]:
# 增加特征 count_day_month_f

In [48]:
def feature_count(data, features):
    feature_name = "count"
    for i in features:
        feature_name += "_" + i
    
    temp = data.groupby(features).size().reset_index().rename(columns={0:feature_name}) # 增加一列count_day_month_f
    data = data.merge(temp, "left", on=features)
    return data, feature_name


ll = []
for f in['campaign', 'contact','default','education','housing','job','loan','marital','poutcome','pdays','previous']:
    data, _ = feature_count(data, ["day", "month", f])
    ll.append(_)
print(ll)

['count_day_month_campaign', 'count_day_month_contact', 'count_day_month_default', 'count_day_month_education', 'count_day_month_housing', 'count_day_month_job', 'count_day_month_loan', 'count_day_month_marital', 'count_day_month_poutcome', 'count_day_month_pdays', 'count_day_month_previous']


In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36169 entries, 0 to 36168
Data columns (total 45 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   ID                         36169 non-null  int64 
 1   age                        36169 non-null  int64 
 2   job                        36169 non-null  object
 3   marital                    36169 non-null  object
 4   education                  36169 non-null  object
 5   default                    36169 non-null  object
 6   balance                    36169 non-null  int64 
 7   housing                    36169 non-null  object
 8   loan                       36169 non-null  object
 9   contact                    36169 non-null  object
 10  day                        36169 non-null  int64 
 11  month                      36169 non-null  object
 12  duration                   36169 non-null  int64 
 13  campaign                   36169 non-null  int64 
 14  pdays 

In [50]:
sparse_feature = ['campaign','contact','default','education','housing','job','loan','marital','month','poutcome']
dense_feature = ['age', 'balance','day','duration','campaign','pdays','previous']

def get_new_columns(name, aggs):
    l = []
    for k in aggs.keys():
        for agg in aggs[k]:
            if str(type(agg)) == "<class 'function'>":
                l.append(namee + "_" + k + "_" + "other")
            else:
                l.append(name + "_" + k + "_" + agg)
    return l

for d in tqdm(sparse_feature):
    aggs = {}
    for s in sparse_feature:
        aggs[s] = ["count", "nunique"]
    for den in dense_feature:
        aggs[den] = ["mean", "max", "min", "std"]
    aggs.pop(d)
    
    temp = data.groupby(d).agg(aggs).reset_index() # agg 执行聚合运算
    temp.columns = [d] + get_new_columns(d, aggs)
    data = pd.merge(data, temp, on = d, how = "left")

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:26<00:00,  2.68s/it]


In [51]:
df_train_columns = data.columns.tolist()

In [52]:
df_train_columns

['ID',
 'age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'y',
 'job_I_D_count',
 'marital_I_D_count',
 'education_I_D_count',
 'default_I_D_count',
 'housing_I_D_count',
 'loan_I_D_count',
 'contact_I_D_count',
 'month_I_D_count',
 'poutcome_I_D_count',
 'age_I_D_count',
 'balance_I_D_count',
 'day_I_D_count',
 'duration_I_D_count',
 'campaign_I_D_count',
 'pdays_I_D_count',
 'previous_I_D_count',
 'count_day_month_campaign',
 'count_day_month_contact',
 'count_day_month_default',
 'count_day_month_education',
 'count_day_month_housing',
 'count_day_month_job',
 'count_day_month_loan',
 'count_day_month_marital',
 'count_day_month_poutcome',
 'count_day_month_pdays',
 'count_day_month_previous',
 'campaign_contact_count',
 'campaign_contact_nunique',
 'campaign_default_count',
 'campaign_default_nunique',
 'campaign_education_count',
 'campaign_education_nun

In [53]:
len(df_train_columns)

483

In [54]:
import numpy as np

from scipy.sparse import csr_matrix # 压缩稀疏矩阵
from sklearn.model_selection import KFold

# 加上原数值特征
num_features = ['job_I_D_count',
 'marital_I_D_count',
 'education_I_D_count',
 'default_I_D_count',
 'housing_I_D_count',
 'loan_I_D_count',
 'contact_I_D_count',
 'month_I_D_count',
 'poutcome_I_D_count',
 'age_I_D_count',
 'balance_I_D_count',
 'day_I_D_count',
 'duration_I_D_count',
 'campaign_I_D_count',
 'pdays_I_D_count',
 'previous_I_D_count',
 'count_day_month_campaign',
 'count_day_month_contact',
 'count_day_month_default',
 'count_day_month_education',
 'count_day_month_housing',
 'count_day_month_job',
 'count_day_month_loan',
 'count_day_month_marital',
 'count_day_month_poutcome',
 'count_day_month_pdays',
 'count_day_month_previous',
 'campaign_contact_count',
 'campaign_contact_nunique',
 'campaign_default_count',
 'campaign_default_nunique',
 'campaign_education_count',
 'campaign_education_nunique',
 'campaign_housing_count',
 'campaign_housing_nunique',
 'campaign_job_count',
 'campaign_job_nunique',
 'campaign_loan_count',
 'campaign_loan_nunique',
 'campaign_marital_count',
 'campaign_marital_nunique',
 'campaign_month_count',
 'campaign_month_nunique',
 'campaign_poutcome_count',
 'campaign_poutcome_nunique',
 'campaign_age_mean',
 'campaign_age_max',
 'campaign_age_min',
 'campaign_age_std',
 'campaign_balance_mean',
 'campaign_balance_max',
 'campaign_balance_min',
 'campaign_balance_std',
 'campaign_day_mean',
 'campaign_day_max',
 'campaign_day_min',
 'campaign_day_std',
 'campaign_duration_mean',
 'campaign_duration_max',
 'campaign_duration_min',
 'campaign_duration_std',
 'campaign_pdays_mean',
 'campaign_pdays_max',
 'campaign_pdays_min',
 'campaign_pdays_std',
 'campaign_previous_mean',
 'campaign_previous_max',
 'campaign_previous_min',
 'campaign_previous_std',
 'contact_campaign_mean',
 'contact_campaign_max',
 'contact_campaign_min',
 'contact_campaign_std',
 'contact_default_count',
 'contact_default_nunique',
 'contact_education_count',
 'contact_education_nunique',
 'contact_housing_count',
 'contact_housing_nunique',
 'contact_job_count',
 'contact_job_nunique',
 'contact_loan_count',
 'contact_loan_nunique',
 'contact_marital_count',
 'contact_marital_nunique',
 'contact_month_count',
 'contact_month_nunique',
 'contact_poutcome_count',
 'contact_poutcome_nunique',
 'contact_age_mean',
 'contact_age_max',
 'contact_age_min',
 'contact_age_std',
 'contact_balance_mean',
 'contact_balance_max',
 'contact_balance_min',
 'contact_balance_std',
 'contact_day_mean',
 'contact_day_max',
 'contact_day_min',
 'contact_day_std',
 'contact_duration_mean',
 'contact_duration_max',
 'contact_duration_min',
 'contact_duration_std',
 'contact_pdays_mean',
 'contact_pdays_max',
 'contact_pdays_min',
 'contact_pdays_std',
 'contact_previous_mean',
 'contact_previous_max',
 'contact_previous_min',
 'contact_previous_std',
 'default_campaign_mean',
 'default_campaign_max',
 'default_campaign_min',
 'default_campaign_std',
 'default_contact_count',
 'default_contact_nunique',
 'default_education_count',
 'default_education_nunique',
 'default_housing_count',
 'default_housing_nunique',
 'default_job_count',
 'default_job_nunique',
 'default_loan_count',
 'default_loan_nunique',
 'default_marital_count',
 'default_marital_nunique',
 'default_month_count',
 'default_month_nunique',
 'default_poutcome_count',
 'default_poutcome_nunique',
 'default_age_mean',
 'default_age_max',
 'default_age_min',
 'default_age_std',
 'default_balance_mean',
 'default_balance_max',
 'default_balance_min',
 'default_balance_std',
 'default_day_mean',
 'default_day_max',
 'default_day_min',
 'default_day_std',
 'default_duration_mean',
 'default_duration_max',
 'default_duration_min',
 'default_duration_std',
 'default_pdays_mean',
 'default_pdays_max',
 'default_pdays_min',
 'default_pdays_std',
 'default_previous_mean',
 'default_previous_max',
 'default_previous_min',
 'default_previous_std',
 'education_campaign_mean',
 'education_campaign_max',
 'education_campaign_min',
 'education_campaign_std',
 'education_contact_count',
 'education_contact_nunique',
 'education_default_count',
 'education_default_nunique',
 'education_housing_count',
 'education_housing_nunique',
 'education_job_count',
 'education_job_nunique',
 'education_loan_count',
 'education_loan_nunique',
 'education_marital_count',
 'education_marital_nunique',
 'education_month_count',
 'education_month_nunique',
 'education_poutcome_count',
 'education_poutcome_nunique',
 'education_age_mean',
 'education_age_max',
 'education_age_min',
 'education_age_std',
 'education_balance_mean',
 'education_balance_max',
 'education_balance_min',
 'education_balance_std',
 'education_day_mean',
 'education_day_max',
 'education_day_min',
 'education_day_std',
 'education_duration_mean',
 'education_duration_max',
 'education_duration_min',
 'education_duration_std',
 'education_pdays_mean',
 'education_pdays_max',
 'education_pdays_min',
 'education_pdays_std',
 'education_previous_mean',
 'education_previous_max',
 'education_previous_min',
 'education_previous_std',
 'housing_campaign_mean',
 'housing_campaign_max',
 'housing_campaign_min',
 'housing_campaign_std',
 'housing_contact_count',
 'housing_contact_nunique',
 'housing_default_count',
 'housing_default_nunique',
 'housing_education_count',
 'housing_education_nunique',
 'housing_job_count',
 'housing_job_nunique',
 'housing_loan_count',
 'housing_loan_nunique',
 'housing_marital_count',
 'housing_marital_nunique',
 'housing_month_count',
 'housing_month_nunique',
 'housing_poutcome_count',
 'housing_poutcome_nunique',
 'housing_age_mean',
 'housing_age_max',
 'housing_age_min',
 'housing_age_std',
 'housing_balance_mean',
 'housing_balance_max',
 'housing_balance_min',
 'housing_balance_std',
 'housing_day_mean',
 'housing_day_max',
 'housing_day_min',
 'housing_day_std',
 'housing_duration_mean',
 'housing_duration_max',
 'housing_duration_min',
 'housing_duration_std',
 'housing_pdays_mean',
 'housing_pdays_max',
 'housing_pdays_min',
 'housing_pdays_std',
 'housing_previous_mean',
 'housing_previous_max',
 'housing_previous_min',
 'housing_previous_std',
 'job_campaign_mean',
 'job_campaign_max',
 'job_campaign_min',
 'job_campaign_std',
 'job_contact_count',
 'job_contact_nunique',
 'job_default_count',
 'job_default_nunique',
 'job_education_count',
 'job_education_nunique',
 'job_housing_count',
 'job_housing_nunique',
 'job_loan_count',
 'job_loan_nunique',
 'job_marital_count',
 'job_marital_nunique',
 'job_month_count',
 'job_month_nunique',
 'job_poutcome_count',
 'job_poutcome_nunique',
 'job_age_mean',
 'job_age_max',
 'job_age_min',
 'job_age_std',
 'job_balance_mean',
 'job_balance_max',
 'job_balance_min',
 'job_balance_std',
 'job_day_mean',
 'job_day_max',
 'job_day_min',
 'job_day_std',
 'job_duration_mean',
 'job_duration_max',
 'job_duration_min',
 'job_duration_std',
 'job_pdays_mean',
 'job_pdays_max',
 'job_pdays_min',
 'job_pdays_std',
 'job_previous_mean',
 'job_previous_max',
 'job_previous_min',
 'job_previous_std',
 'loan_campaign_mean',
 'loan_campaign_max',
 'loan_campaign_min',
 'loan_campaign_std',
 'loan_contact_count',
 'loan_contact_nunique',
 'loan_default_count',
 'loan_default_nunique',
 'loan_education_count',
 'loan_education_nunique',
 'loan_housing_count',
 'loan_housing_nunique',
 'loan_job_count',
 'loan_job_nunique',
 'loan_marital_count',
 'loan_marital_nunique',
 'loan_month_count',
 'loan_month_nunique',
 'loan_poutcome_count',
 'loan_poutcome_nunique',
 'loan_age_mean',
 'loan_age_max',
 'loan_age_min',
 'loan_age_std',
 'loan_balance_mean',
 'loan_balance_max',
 'loan_balance_min',
 'loan_balance_std',
 'loan_day_mean',
 'loan_day_max',
 'loan_day_min',
 'loan_day_std',
 'loan_duration_mean',
 'loan_duration_max',
 'loan_duration_min',
 'loan_duration_std',
 'loan_pdays_mean',
 'loan_pdays_max',
 'loan_pdays_min',
 'loan_pdays_std',
 'loan_previous_mean',
 'loan_previous_max',
 'loan_previous_min',
 'loan_previous_std',
 'marital_campaign_mean',
 'marital_campaign_max',
 'marital_campaign_min',
 'marital_campaign_std',
 'marital_contact_count',
 'marital_contact_nunique',
 'marital_default_count',
 'marital_default_nunique',
 'marital_education_count',
 'marital_education_nunique',
 'marital_housing_count',
 'marital_housing_nunique',
 'marital_job_count',
 'marital_job_nunique',
 'marital_loan_count',
 'marital_loan_nunique',
 'marital_month_count',
 'marital_month_nunique',
 'marital_poutcome_count',
 'marital_poutcome_nunique',
 'marital_age_mean',
 'marital_age_max',
 'marital_age_min',
 'marital_age_std',
 'marital_balance_mean',
 'marital_balance_max',
 'marital_balance_min',
 'marital_balance_std',
 'marital_day_mean',
 'marital_day_max',
 'marital_day_min',
 'marital_day_std',
 'marital_duration_mean',
 'marital_duration_max',
 'marital_duration_min',
 'marital_duration_std',
 'marital_pdays_mean',
 'marital_pdays_max',
 'marital_pdays_min',
 'marital_pdays_std',
 'marital_previous_mean',
 'marital_previous_max',
 'marital_previous_min',
 'marital_previous_std',
 'month_campaign_mean',
 'month_campaign_max',
 'month_campaign_min',
 'month_campaign_std',
 'month_contact_count',
 'month_contact_nunique',
 'month_default_count',
 'month_default_nunique',
 'month_education_count',
 'month_education_nunique',
 'month_housing_count',
 'month_housing_nunique',
 'month_job_count',
 'month_job_nunique',
 'month_loan_count',
 'month_loan_nunique',
 'month_marital_count',
 'month_marital_nunique',
 'month_poutcome_count',
 'month_poutcome_nunique',
 'month_age_mean',
 'month_age_max',
 'month_age_min',
 'month_age_std',
 'month_balance_mean',
 'month_balance_max',
 'month_balance_min',
 'month_balance_std',
 'month_day_mean',
 'month_day_max',
 'month_day_min',
 'month_day_std',
 'month_duration_mean',
 'month_duration_max',
 'month_duration_min',
 'month_duration_std',
 'month_pdays_mean',
 'month_pdays_max',
 'month_pdays_min',
 'month_pdays_std',
 'month_previous_mean',
 'month_previous_max',
 'month_previous_min',
 'month_previous_std',
 'poutcome_campaign_mean',
 'poutcome_campaign_max',
 'poutcome_campaign_min',
 'poutcome_campaign_std',
 'poutcome_contact_count',
 'poutcome_contact_nunique',
 'poutcome_default_count',
 'poutcome_default_nunique',
 'poutcome_education_count',
 'poutcome_education_nunique',
 'poutcome_housing_count',
 'poutcome_housing_nunique',
 'poutcome_job_count',
 'poutcome_job_nunique',
 'poutcome_loan_count',
 'poutcome_loan_nunique',
 'poutcome_marital_count',
 'poutcome_marital_nunique',
 'poutcome_month_count',
 'poutcome_month_nunique',
 'poutcome_age_mean',
 'poutcome_age_max',
 'poutcome_age_min',
 'poutcome_age_std',
 'poutcome_balance_mean',
 'poutcome_balance_max',
 'poutcome_balance_min',
 'poutcome_balance_std',
 'poutcome_day_mean',
 'poutcome_day_max',
 'poutcome_day_min',
 'poutcome_day_std',
 'poutcome_duration_mean',
 'poutcome_duration_max',
 'poutcome_duration_min',
 'poutcome_duration_std',
 'poutcome_pdays_mean',
 'poutcome_pdays_max',
 'poutcome_pdays_min',
 'poutcome_pdays_std',
 'poutcome_previous_mean',
 'poutcome_previous_max',
 'poutcome_previous_min',
 'poutcome_previous_std'] + num_features

# one_hot特征备用
one_hot_feature =['job', 'marital','education','default','housing','loan','contact','month','poutcome']

data["new_con"] = data["job"].astype(str)
for i in ["marital", "education", "contact", "month", "poutcome"]:
    data["new_con"] = data["new_con"].astype(str) + "_" + data[i].astype(str)
data["new_con"] = data["new_con"].apply(lambda x: " ".join(x.split("_")))

data.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,...,poutcome_duration_std,poutcome_pdays_mean,poutcome_pdays_max,poutcome_pdays_min,poutcome_pdays_std,poutcome_previous_mean,poutcome_previous_max,poutcome_previous_min,poutcome_previous_std,new_con
0,1,43,management,married,tertiary,no,291,yes,no,unknown,...,260.008036,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992,management married tertiary unknown may unknown
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,...,241.741093,226.557966,838,1,123.825801,4.019661,275,1,8.374317,technician divorced primary cellular apr other
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,...,260.008036,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992,admin. married secondary cellular jul unknown
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,...,260.008036,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992,management single secondary cellular jul unknown
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,...,260.008036,-0.952528,528,-1,4.225828,0.000575,7,0,0.052992,technician divorced secondary unknown may unknown


In [77]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html
import pandas as pd

for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])

vector_feature = ["new_con"]

train = data[data.y != -1] # 训练集
test = data[data.y == -1] # 测试集
res = test[["ID"]]
# print(res)

train_x = train[num_features]
test_x = test[num_features]
# print(type(train_x))

# OneHotEncoder
enc = OneHotEncoder()
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a = enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1,1))
    train_x = sparse.hstack((train_x, train_a), "csr")
    test_x = sparse.hstack((test_x, test_a), "csr")

# CountVectorizer
cv = CountVectorizer()
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
    test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a), "csr")
    test_x = sparse.hstack((test_x, test_a), "csr")

# KFold
n_splits = 5
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
clf = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )

train_x = train_x.toarray()
test_x = test_x.toarray()
train_y = train["y"]
print(train_x.shape)
res["pred"] = 0
for train_idx, val_idx in kfold.split(train_x):
    print(train_idx, val_idx)
    print(train_idx.shape,val_idx.shape)
    print(type(train_x))
    print(type(train_idx), type(val_idx))
    clf.random_state = clf.random_state + 1
    train_x1 = train_x[train_idx.tolist()]
    train_y1 = train_y[train_idx.tolist()]
    test_x1 = train_x[val_idx.tolist()]
    test_y1 = train_y[val_idx.tolist()]
    clf.fit(train_x1, train_y1, eval_set=[(train_x1, train_y1), (test_x1, test_y1)], eval_metric="auc", early_stopping_rounds=100)
    res["pred"] += clf.predict_proba(test_x)[:,1]
    
res["pred"] = res["pred"] / 5
res.to_csv("./basesubmission.csv", index=False)

(25317, 553)
[    0     1     2 ... 25314 25315 25316] [   17    29    30 ... 25304 25309 25312]
(20253,) (5064,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


F:\anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
F:\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[1]	training's auc: 0.911335	valid_1's auc: 0.894299
[2]	training's auc: 0.927587	valid_1's auc: 0.907373
[3]	training's auc: 0.93151	valid_1's auc: 0.919754
[4]	training's auc: 0.93295	valid_1's auc: 0.918178
[5]	training's auc: 0.933209	valid_1's auc: 0.918355
[6]	training's auc: 0.932957	valid_1's auc: 0.921222
[7]	training's auc: 0.933254	valid_1's auc: 0.92171
[8]	training's auc: 0.932513	valid_1's auc: 0.922557
[9]	training's auc: 0.931583	valid_1's auc: 0.921975
[10]	training's auc: 0.930039	valid_1's auc: 0.91946
[11]	training's auc: 0.932648	valid_1's auc: 0.922066
[12]	training's auc: 0.934465	valid_1's auc: 0.923862
[13]	training's auc: 0.937639	valid_1's auc: 0.92817
[14]	training's auc: 0.939428	valid_1's auc: 0.930172
[15]	training's auc: 0.940364	valid_1's auc: 0.931051
[16]	training's auc: 0.941223	valid_1's auc: 0.931508
[17]	training's auc: 0.942205	valid_1's auc: 0.932545
[18]	training's auc: 0.942795	valid_1's auc: 0.932939
[19]	training's auc: 0.943217	valid_1's au

[164]	training's auc: 0.961513	valid_1's auc: 0.938359
[165]	training's auc: 0.961619	valid_1's auc: 0.938393
[166]	training's auc: 0.961681	valid_1's auc: 0.938339
[167]	training's auc: 0.961783	valid_1's auc: 0.938461
[168]	training's auc: 0.961856	valid_1's auc: 0.93844
[169]	training's auc: 0.961941	valid_1's auc: 0.938543
[170]	training's auc: 0.962039	valid_1's auc: 0.938542
[171]	training's auc: 0.962111	valid_1's auc: 0.938526
[172]	training's auc: 0.962209	valid_1's auc: 0.938614
[173]	training's auc: 0.962327	valid_1's auc: 0.938657
[174]	training's auc: 0.962409	valid_1's auc: 0.938664
[175]	training's auc: 0.962503	valid_1's auc: 0.938642
[176]	training's auc: 0.962565	valid_1's auc: 0.938658
[177]	training's auc: 0.962642	valid_1's auc: 0.938589
[178]	training's auc: 0.962745	valid_1's auc: 0.938606
[179]	training's auc: 0.962832	valid_1's auc: 0.93867
[180]	training's auc: 0.962929	valid_1's auc: 0.93868
[181]	training's auc: 0.96302	valid_1's auc: 0.938662
[182]	training

[326]	training's auc: 0.97422	valid_1's auc: 0.939867
[327]	training's auc: 0.974293	valid_1's auc: 0.939886
[328]	training's auc: 0.974351	valid_1's auc: 0.939899
[329]	training's auc: 0.974407	valid_1's auc: 0.939897
[330]	training's auc: 0.974474	valid_1's auc: 0.939909
[331]	training's auc: 0.974551	valid_1's auc: 0.939927
[332]	training's auc: 0.974586	valid_1's auc: 0.939914
[333]	training's auc: 0.97467	valid_1's auc: 0.939933
[334]	training's auc: 0.974739	valid_1's auc: 0.939951
[335]	training's auc: 0.974795	valid_1's auc: 0.93996
[336]	training's auc: 0.974843	valid_1's auc: 0.939982
[337]	training's auc: 0.974913	valid_1's auc: 0.939991
[338]	training's auc: 0.974976	valid_1's auc: 0.940002
[339]	training's auc: 0.975047	valid_1's auc: 0.939979
[340]	training's auc: 0.97512	valid_1's auc: 0.939971
[341]	training's auc: 0.975202	valid_1's auc: 0.939947
[342]	training's auc: 0.975278	valid_1's auc: 0.939939
[343]	training's auc: 0.975346	valid_1's auc: 0.939911
[344]	training

...

F:\anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
F:\anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "


[    0     1     2 ... 25312 25314 25316] [    9    11    13 ... 25306 25313 25315]
(20254,) (5063,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'> <class 'numpy.ndarray'>
[1]	training's auc: 0.914243	valid_1's auc: 0.892354
[2]	training's auc: 0.926677	valid_1's auc: 0.909745
[3]	training's auc: 0.935072	valid_1's auc: 0.92139
[4]	training's auc: 0.936146	valid_1's auc: 0.922469
[5]	training's auc: 0.938642	valid_1's auc: 0.923549
[6]	training's auc: 0.93854	valid_1's auc: 0.924588
[7]	training's auc: 0.93955	valid_1's auc: 0.92565
[8]	training's auc: 0.94081	valid_1's auc: 0.926418
[9]	training's auc: 0.940434	valid_1's auc: 0.926531
[10]	training's auc: 0.941762	valid_1's auc: 0.927447
[11]	training's auc: 0.942381	valid_1's auc: 0.927788
[12]	training's auc: 0.942901	valid_1's auc: 0.928022
[13]	training's auc: 0.943023	valid_1's auc: 0.927927
[14]	training's auc: 0.943428	valid_1's auc: 0.92776
[15]	training's auc: 0.943172	valid_1's auc: 0.92738
[16]	training's auc: 0.943574	val

[149]	training's auc: 0.961559	valid_1's auc: 0.935255
[150]	training's auc: 0.961676	valid_1's auc: 0.935277
[151]	training's auc: 0.961786	valid_1's auc: 0.935297
[152]	training's auc: 0.961882	valid_1's auc: 0.935254
[153]	training's auc: 0.961981	valid_1's auc: 0.935239
[154]	training's auc: 0.962078	valid_1's auc: 0.935164
[155]	training's auc: 0.962159	valid_1's auc: 0.935156
[156]	training's auc: 0.962294	valid_1's auc: 0.935164
[157]	training's auc: 0.962405	valid_1's auc: 0.935179
[158]	training's auc: 0.962504	valid_1's auc: 0.935166
[159]	training's auc: 0.962634	valid_1's auc: 0.935184
[160]	training's auc: 0.962697	valid_1's auc: 0.935183
[161]	training's auc: 0.962773	valid_1's auc: 0.935192
[162]	training's auc: 0.962852	valid_1's auc: 0.935175
[163]	training's auc: 0.962964	valid_1's auc: 0.935208
[164]	training's auc: 0.963032	valid_1's auc: 0.93523
[165]	training's auc: 0.963142	valid_1's auc: 0.935197
[166]	training's auc: 0.963236	valid_1's auc: 0.935219
[167]	train

[301]	training's auc: 0.973935	valid_1's auc: 0.935405
[302]	training's auc: 0.974011	valid_1's auc: 0.93538
[303]	training's auc: 0.97406	valid_1's auc: 0.935427
[304]	training's auc: 0.97412	valid_1's auc: 0.935452
[305]	training's auc: 0.974199	valid_1's auc: 0.935466
[306]	training's auc: 0.974272	valid_1's auc: 0.935488
[307]	training's auc: 0.974367	valid_1's auc: 0.935479
[308]	training's auc: 0.974428	valid_1's auc: 0.935457
[309]	training's auc: 0.974476	valid_1's auc: 0.935457
[310]	training's auc: 0.974546	valid_1's auc: 0.935449
[311]	training's auc: 0.974615	valid_1's auc: 0.935442
[312]	training's auc: 0.974675	valid_1's auc: 0.935457
[313]	training's auc: 0.97475	valid_1's auc: 0.935456
[314]	training's auc: 0.974812	valid_1's auc: 0.935493
[315]	training's auc: 0.974881	valid_1's auc: 0.935479
[316]	training's auc: 0.974956	valid_1's auc: 0.935482
[317]	training's auc: 0.975012	valid_1's auc: 0.935405
[318]	training's auc: 0.975067	valid_1's auc: 0.935402
[319]	training

F:\anaconda3\lib\site-packages\ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
F:\anaconda3\lib\site-packages\ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
